In [1]:
import altair as alt
import polars as pl
from pathlib import Path
from vega_datasets import data
import geopandas as gpd
import pandas as pd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
# Load data set
df = pl.read_csv("data/FoodImports.csv", ignore_errors=True)

# Clean data set
df_clean = (
    df
    .filter(
        ~pl.col("Country").str.contains("WORLD|Rest of World|Quantity")
        & (pl.col("UOM") == "Million $")
        & pl.col("FoodValue").is_not_null()
        & pl.col("YearNum").is_not_null()
    )
    .filter(~pl.col("Commodity").str.starts_with("Total"))
)
df_clean_b = df_clean.with_columns(
    (pl.col("FoodValue") / 1000).alias("FoodValue_B")
)

In [3]:
chart_import_trend = (
    alt.Chart(df_clean_b)
    .mark_line(point=True)
    .encode(
        x=alt.X("YearNum:O", title="Year"), 
        y=alt.Y(
            "sum(FoodValue_B):Q", 
            title="Total Import Value (Billion $)"
        )
    )
    .properties(title={
                "text": "Total US Food Imports Over Time (1999 - 2024)",
                "subtitle": "Values shown in nominal U.S. dollars (not adjusted for inflation)"
                }
    )
)

chart_import_trend

alt.Chart(...)

In [4]:
df_country_totals = (
    df_clean_b
    .group_by("Country")
    .agg(pl.sum("FoodValue_B").alias("TotalImports_B"))
    .sort("TotalImports_B", descending=True)
)

df_top8 = df_country_totals.head(8)

custom_colors = {
    "MEXICO": "#006847",
    "CANADA": "#C8102E",
    "CHINA": "#EE1C25",
    "INDIA": "#FF9933",
    "FRANCE": "#0055A4",
    "ITALY": "#009246",
    "CHILE": "#0033A0",
    "INDONESIA": "#D00000",
}

chart_top_exporting_countries = (
    alt.Chart(df_top8)
    .mark_bar()
    .encode(
        x=alt.X("TotalImports_B:Q", title="Accumulated Total Food Imports from 1999 to 2024 (Billion $)"),
        y=alt.Y("Country:N", sort='-x', title="Country"),
        color=alt.Color(
            "Country:N",
            scale=alt.Scale(
                domain=list(custom_colors.keys()),
                range=list(custom_colors.values())
            ),
            legend=None
        )
    )
    .properties(
        title=f"Top Exporting Countries to the US"
    )
)

chart_top_exporting_countries

alt.Chart(...)

In [5]:
df_can_mex = (
    df_clean_b
    .filter(pl.col("Country").is_in(["CANADA", "MEXICO"]))
    .group_by(["YearNum", "Country"])
    .agg(pl.sum("FoodValue_B").alias("TotalImports_B"))
    .sort(["Country", "YearNum"])
)


chart_can_mex = (
    alt.Chart(df_can_mex)
    .mark_line(point=True)
    .encode(
        x=alt.X("YearNum:O", title="Year"),
        y=alt.Y("TotalImports_B:Q", title="Total Import Value (Billion $)"),
        color=alt.Color("Country:N", title="Country", scale=alt.Scale(domain=["CANADA", "MEXICO"], range=["#C8102E", "#006847"])),
    )
    .properties(
        title={
            "text": "Canada vs Mexico: U.S. Food Import Trends Over Time (1999 - 2024)",
            "subtitle": "Values shown in nominal U.S. dollars (not adjusted for inflation)"
        }
    )
)

rule = (
    alt.Chart(alt.Data(values=[{"YearNum": 2016}]))
    .mark_rule(color="gray")
    .encode(x="YearNum:O")
)

chart_can_mex + rule

alt.LayerChart(...)

🇺🇸 Pre-2016: Stable North American Trade under NAFTA (1994–2016)

NAFTA, signed in 1994, virtually eliminated tariffs on most agricultural products between the U.S., Mexico, and Canada.

Result: food trade grew steadily —

Canada remained the top exporter of grains, meats, and processed food,

Mexico specialized in fresh produce, beer.

Policy environment: predictable and pro-free trade; minimal political disruption.

🧩 In your chart: relatively smooth, parallel growth in U.S. imports from both countries through 2016.

🏛 2016–2020: Trump Administration — “America First” Trade Policy
🔹 2016 Campaign & Early 2017

Trump campaigned on NAFTA being a “bad deal” and promised to renegotiate or withdraw.

January 2017: U.S. withdrew from TPP (Trans-Pacific Partnership) — signaling a pivot from multilateral to bilateral trade.

April 2017: ordered a NAFTA renegotiation.

Created short-term uncertainty: some Canadian exports slowed, and importers diversified toward Mexico.

🔹 2017–2018: NAFTA Renegotiation Begins

The administration imposed tariffs on steel and aluminum (25% and 10%) on Canada and Mexico.

Canada retaliated with tariffs on U.S. dairy and food products.

Result: rising tension and temporary slowdown in some bilateral food flows.

Negotiations aimed to favor U.S. farmers, but the uncertainty caused many firms to shift sourcing to Mexico (cheaper, flexible supply).

🔹 2018–2019: USMCA Agreement & China Trade War

USMCA signed in late 2018, ratified 2019, took effect July 2020.

Expanded U.S. dairy access to Canada (a small gain for U.S. exporters).

Preserved duty-free trade for most agri-food.

Added stricter rules of origin and labor standards, pushing manufacturers (esp. processed foods) to relocate some production to Mexico.

Simultaneously: U.S.–China trade war began in 2018.

Tariffs on Chinese foods and ingredients (seafood, soy, garlic, etc.)

U.S. importers substituted Chinese goods with Mexican/Canadian suppliers — especially Mexico due to cost advantage.

🧩 In your chart: 2018–2019 often marks an inflection — Mexico’s export line climbing, Canada’s flattening.

🔹 2020: COVID-19 & USMCA Implementation

USMCA went into effect July 1 2020, solidifying North American integration.

Pandemic border controls temporarily disrupted Canadian trucking and logistics.

Mexico’s fresh produce sector rebounded faster, keeping exports high while Canadian shipments lagged due to cold-storage and labor issues.

🧩 In your chart: short dip or divergence around 2020–2021, then Mexico widening its lead.

📈 2021–2024: Biden Administration — Stability & Re-regionalization

Biden maintained USMCA and tariff structures; no return to TPP.

Focus shifted to supply chain resilience and “friend-shoring” — strengthening trade within the Americas.

Inflation and strong dollar increased nominal import values, even if real quantities were stable.

Canada’s exports stabilized; Mexico continued gradual rise in categories like produce, beverages, and processed foods.

In [10]:
top10_by_country = (
    df_clean_b
    .filter(pl.col("Country").is_in(["MEXICO", "CANADA"]))
    .group_by(["Country", "Commodity"])
    .agg(pl.col("FoodValue_B").sum().alias("TotalImports_B"))
    .sort(["Country", "TotalImports_B"], descending=[True, True])
    .group_by("Country")
    .head(10)
)

chart_top10_commodities = (
    alt.Chart(top10_by_country)
    .mark_bar()
    .encode(
        x=alt.X("TotalImports_B:Q", title="Total Food Imports (Billion $)"),
        y=alt.Y("Commodity:N", sort='-x', title="Commodity"),
        color=alt.Color("Country:N", scale=alt.Scale(domain=["CANADA", "MEXICO"], range=["#C8102E", "#006847"])),
        facet=alt.Facet("Country:N", columns=2, title=None)
    )
    .properties(title="Accumulated Top 10 Food Commodities Imported from Canada and Mexico (1999 - 2024)")
)

chart_top10_commodities

alt.Chart(...)

🇲🇽 Mexico (Right side)

Dominated by fresh vegetables, fruits, and beer/beverages → clear agricultural & manufacturing advantage.

Strong presence in prepared or preserved vegetables — fits post-2016 trend of processed exports rising.

Cereal & bakery foods show smaller values — Mexico’s exports are more fresh/seasonal.

🇨🇦 Canada (Left side)

Dominated by fresh/chilled meats, grains, and prepared foods.

Stronger presence in chocolate, refined oils, and malt beer → indicates more industrial/processed food trade.

Suggests a pattern: Mexico = fresh agricultural, Canada = processed / industrial food.

In [7]:
df_with_period = df_clean_b.with_columns(
    pl.when(pl.col("YearNum") <= 2016)
    .then(pl.lit("Before_2016"))
    .otherwise(pl.lit("After_2016"))
    .alias("Period")
)

df_agg = (
    df_with_period
    .filter(pl.col("Country").is_in(["MEXICO", "CANADA"]))
    .group_by(["Country", "Commodity", "Period"])
    .agg(pl.col("FoodValue_B").sum().alias("TotalValue_B"))
)

df_pivot = (
    df_agg
    .pivot(index=["Country", "Commodity"], columns="Period", values="TotalValue_B")
)

df_shift = df_pivot.with_columns(
    (pl.col("After_2016").fill_null(0) - pl.col("Before_2016").fill_null(0)).alias("Change")
)

df_shift_top10 = (
    df_shift
    .with_columns(pl.col("Change").abs().alias("AbsChange"))
    .sort("AbsChange", descending=True)
    .head(10)
)

chart_shift_top10 = (
    alt.Chart(df_shift_top10)
    .mark_bar()
    .encode(
        x=alt.X("Change:Q", title="Change in Import Value (After minus Before 2016, $B)"),
        y=alt.Y("Commodity:N", sort='x'),
        color=alt.Color(
            "Country:N",
            scale=alt.Scale(domain=["MEXICO", "CANADA"],
                            range=["#006847", "#C8102E"])
        ),
    )
    .properties(title="Shift in U.S. Food Imports by Commodity After 2016")
)

chart_shift_top10


/var/folders/j1/7ggvvb311jl0d3c1xxls5rrh0000gn/T/ipykernel_9811/2200112199.py:16: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  df_agg


alt.Chart(...)

🇲🇽 Mexico

Large positive shifts for:

Fresh fruits and vegetables, tree nuts, beer, bakery foods, etc.

These are labor-intensive or climate-dependent products that the U.S. increasingly sources from Mexico instead of Asia.

→ Reflects supply-chain “nearshoring” and year-round produce demand after trade tensions with China.

🇨🇦 Canada

Positive shifts for:

Red meats, dairy, cereal products → aligns with Canada’s strong agri-processing sector.

Declines for some fresh goods — climate limits Canada’s seasonal crops.

→ Suggests specialization in processed and animal-based exports rather than produce.

🧩 Takeaway:

After 2016, U.S. food imports realigned within North America —

Mexico gained ground in produce, beverages, and light manufacturing.

Canada consolidated its position in processed and high-value food commodities.

So this chart is visual evidence of supply-chain reorganization under post-2016 trade policy — fewer imports from China → increased intra-NAFTA trade.

After 2016, the U.S. government imposed 25% tariffs on most goods from Mexico and Canada,
except:

Canadian oil and energy exports → only 10% tariff.

Mexican energy exports → full 25%.

That means:

Goods with energy-related inputs (e.g., food processing, fertilizers, transportation-heavy commodities)
→ will show different responses between the two partners.

Price-sensitive agricultural imports → U.S. may import less after 2016.

Commodities exempt or strategically important (like oil-linked categories) → likely stable or even grew.

| Observation                                                                 | Likely Interpretation                                                                                    | Policy Connection                                       |
| --------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------- | ------------------------------------------------------- |
| 🇲🇽 Fresh fruits, vegetables, beer, bakery goods → **increase after 2016** | Despite tariffs, U.S. demand remained high — food perishability and proximity make Mexico indispensable. | *Tariffs had limited effect; “nearshoring resilience.”* |
| 🇨🇦 Red meats, dairy, cereals → **increase**                               | Processed and energy-intensive goods stayed competitive — benefited from **lower (10%) energy tariffs**. | *Partial exemption = advantage to Canadian processors.* |
| 🇲🇽 Processed or energy-dependent goods → **decline**                      | Higher transportation and energy costs from 25% tariff plus energy-price rise.                           | *Direct negative impact of tariff policy.*              |


In [11]:
df_shift = df_pivot.with_columns(
    (pl.col("After_2016").fill_null(0) - pl.col("Before_2016").fill_null(0)).alias("Change_B")
)

energy_intensive = [
    "Cereal and bakery foods", "Prepared meats", "Processed meats",
    "Dairy products", "Refined vegetable oils", "Chocolate",
    "Frozen red meats and parts", "Cocoa paste, butter, and powder",
    "Sauces, soups, prepared foods", "Beer and malt"
]

df_shift = df_shift.with_columns(
    pl.when(pl.col("Commodity").is_in(energy_intensive))
    .then(pl.lit("High tariff exposure"))
    .otherwise(pl.lit("Low tariff exposure"))
    .alias("TariffSensitivity")
)

df_shift_pivot = (
    df_shift
    .pivot(values="Change_B", index="Commodity", columns="Country")
    .rename({"CANADA": "Change_CANADA", "MEXICO": "Change_MEXICO"})
    .join(df_shift.select(["Commodity", "TariffSensitivity"]), on="Commodity", how="left")
)

labels = alt.Chart(pd.DataFrame({
    "x": [24, 24, 24, 24],
    "y": [7, 6, -7, -6],
    "text": [
        "↑ Both increased (Integrated goods)",
        "↑ Canada gain (Energy-intensive)",
        "↓ Mexico gain (Perishables)",
        "↓ Both declined (Tariff-sensitive)"
    ]
})).mark_text(color="gray", fontSize=11).encode(x="x:Q", y="y:Q", text="text:N")

rule_x = alt.Chart(pd.DataFrame({"x": [0]})).mark_rule(strokeDash=[5,3], color="gray").encode(x="x:Q")
rule_y = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(strokeDash=[5,3], color="gray").encode(y="y:Q")

chart_quadrant = (
    alt.Chart(df_shift_pivot)
    .mark_circle(size=90, opacity=0.7)
    .encode(
        x=alt.X("Change_MEXICO:Q", title="Change After 2016 ($B, Mexico)"),
        y=alt.Y("Change_CANADA:Q", title="Change After 2016 ($B, Canada)"),
        color=alt.Color("TariffSensitivity:N",
                        scale=alt.Scale(domain=["High tariff exposure","Low tariff exposure"],
                                        range=["#ff7f0e","#1f77b4"])),
        tooltip=["Commodity","Change_MEXICO","Change_CANADA","TariffSensitivity"]
    )
    .properties(title="Commodity Growth Matrix: Mexico vs Canada (After 2016)", width=820, height=520)
)

highlight_items = [
    "Wheat and products", "Fresh vegetables", "Malt beer", "Cereal and bakery foods",
    "Prepared meats", "Fresh or chilled fruit", "Refined vegetable oils",
    "Shellfish, fresh or frozen", "Bovine animals, live", "Fresh or chilled red meats",
    "Liquors and liqueurs"
]

text = (
    alt.Chart(df_shift_pivot.filter(pl.col("Commodity").is_in(highlight_items)))
    .mark_text(align="left", dx=5, dy=-5, fontSize=12)
    .encode(x="Change_MEXICO:Q", y="Change_CANADA:Q", text="Commodity:N")
)

chart_final = chart_quadrant + rule_x + rule_y + labels + text
chart_final


/var/folders/j1/7ggvvb311jl0d3c1xxls5rrh0000gn/T/ipykernel_9811/2671584033.py:20: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  df_shift


alt.LayerChart(...)

In [13]:
df_share = (
    df_clean_b
    .filter(pl.col("Country").is_in(["MEXICO","CANADA", "CHINA"]))
    .group_by(["YearNum", "Country"])
    .agg(pl.sum("FoodValue_B").alias("TotalValue_B"))
)

total_per_year = df_share.group_by("YearNum").agg(pl.sum("TotalValue_B").alias("TotalYearValue"))
df_share = df_share.join(total_per_year, on="YearNum")
df_share = df_share.with_columns((pl.col("TotalValue_B") / pl.col("TotalYearValue") * 100).alias("SharePct"))

chart_share = (
    alt.Chart(df_share)
    .mark_area(opacity=0.8)
    .encode(
        x=alt.X("YearNum:O", title="Year"),
        y=alt.Y("SharePct:Q", title="Share of Total Imports (%)", stack="normalize"),
        color=alt.Color("Country:N", scale=alt.Scale(domain=["CANADA","MEXICO", "CHINA"], range=["#C8102E","#006847", "#EE1C25"]))
    )
    .properties(title="Share of U.S. Food Imports by Country (Canada vs Mexico, 1999–2024)")
)
chart_share


alt.Chart(...)